# Phương trình tay máy

In [ ]:
import numpy as np
a_s = [0, 45, 107]


a_r0 = (a_s[0]*np.pi)/180
a_r1 = (a_s[1] *np.pi)/180
a_r2 = (a_s[2]*np.pi)/180
print("input :", a_r0, a_r1, a_r2)
q1 = a_r0
q2 = a_r1
q3 = a_r2
L0 = 7.5
L1 = 8
L2 = 11.5



x = L1*np.cos(q2)*np.cos(q1) + L2*np.cos(q3-q2)*np.cos(q1)
y = L1*np.cos(q2)*np.sin(q1) + L2*np.cos(q3-q2)*np.sin(q1)
z = L1*np.sin(q2) - L2*np.sin(q3-q2) + L0

print(x, y, z)

x, y, z = 12, 2, 3
# x 4 - 13
# y = 4 - 10
# z = 0 - 5

a_1 = np.arctan(y/x)

n = z-L0
m = x*np.cos(a_1) + y*np.sin(a_1)

a_3 = np.arccos((np.power(m,2) + np.power(n,2) - np.power(L1,2) - np.power(L2,2))/(2*L1*L2))

u = L1 + L2*np.cos(a_3)
v = L2 * np.sin(a_3)

a_2 = np.arccos((u*m-v*n)/(u**2 + v**2))

a_r0 = np.pi/2 - a_1
a_r1 = a_2 
a_r2 = a_3 - a_2

print(a_r0*180/np.pi, a_r1*180/np.pi, a_r2*180/np.pi)

# Xử lý dữ liệu ảnh

# Xử lý dữ liệu bảng 

In [ ]:
import pandas as pd
import numpy as np
df = pd.read_csv("D:/User/DLBot/scripts/data/angles.csv")
# df.head()
data = np.array(df)
X = np.array(data[:,0:8], dtype=np.double)/400
y = np.array(data[:,8:12], dtype=np.double)*np.pi/180

In [1]:
L0 = 7.5
L1 = 8
L2 = 11.5
def donghocthuan(q):
    x = L1*np.cos(q[1])*np.cos(q[0]) + L2*np.cos(q[2]-q[1])*np.cos(q[0])
    y = L1*np.cos(q[1])*np.sin(q[0]) + L2*np.cos(q[2]-q[1])*np.sin(q[0])
    z = L1*np.sin(q[1]) - L2*np.sin(q[2]-q[1]) + L0
    return x, y, z, q[3]

def center_box(box):
    x, y = box[0]+(box[2] - box[0])/2, box[1]+(box[3] - box[1])/2
    w, h = (box[2] - box[0]), (box[3] - box[1])
    return x, y, w, h

In [ ]:
data_create = []
for idx, dta in enumerate(X):
    box_1, box_2 = dta[:4], dta[4:]
    # x, y, w, h = center_box(box_1)
    # print(x,y,w,h)
    # print(idx)
    data_frame = np.hstack(
                        [center_box(box_1), center_box(box_2), donghocthuan(y[idx])])
    data_create.append(data_frame)
# df = pd.read_csv("D:/User/DLBot/scripts/data/angles_pro.csv")
df2 = pd.DataFrame(data_create, columns=['x1','y1','w1','h1','x2','y2','w2','h2','a1','a2','a3','a4'])
# df2 = pd.concat([df, df2])
df2.to_csv(
    "D:/User/DLBot/scripts/data/angles_pro.csv", index=False)
data_create

In [ ]:
def create_batch_data(Train_X, Train_y, batch):
    X, y = [], []
    XX, yy = [], []
    j=0
    while j < len(Train_y):
        X.append(Train_X[j])
        y.append(Train_y[j])
        if (j+1)%batch==0:
            XX.append(X)
            yy.append(y)
            X, y = [], []
        j+=1
    return np.array(XX), np.array(yy)

# Training

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from model_ANN import ANN
import torch.nn as nn
import tqdm
import torch

df = pd.read_csv("D:/User/DLBot/scripts/data/angles_pro.csv")
# df.head()
data = np.array(df)
X = np.array(data[:,0:8], dtype=np.double)
y = np.array(data[:,8:12], dtype=np.double)

Train_X, Val_X, Train_y, Val_y = train_test_split(X, y, test_size=0.2)
X_train, y_train = create_batch_data(Train_X, Train_y, 64)
X_val, y_val = create_batch_data(Val_X, Val_y, 64)

In [ ]:
def study(model, X, y, optimizer, losses, device):
    model.train()
    train_loss = 0
    # counter = 0
    # print ("@@@ Start train model @@@")
    for i, data in enumerate(X):
        # counter += 1
        image, label = torch.tensor(data,dtype=torch.float32), torch.tensor(y[i],dtype=torch.float32)
        image = image.to(device)
        label =label.to(device)
        optimizer.zero_grad()
        label_pre = model(image)
        # print(label_pre.shape)
        loss = losses(label_pre, label)
        train_loss += loss.item()
        # print(train_loss)
        loss.backward()
        optimizer.step()
        # print(preds)
    epoch_loss = train_loss / len(X)

    return epoch_loss


In [ ]:
def val(model, X, y, losses, device):
    val_loss = 0
    # counter = 0
    # print ("@@@ Start train model @@@")
    for i, data in enumerate(X):
        # counter += 1
        image, label = torch.tensor(data,dtype=torch.float32), torch.tensor(y[i],dtype=torch.float32)
        image = image.to(device)
        label =label.to(device)
        label_pre = model(image)
        # print(label_pre.shape)
        loss = losses(label_pre, label)
        val_loss += loss.item()
        # print(train_loss)
        # loss.backward()
        # print(preds)
    epoch_loss = val_loss / len(X)

    return epoch_loss

In [ ]:
import torch.optim as optim

PATH = "D:/User/DLBot/scripts/model/model_auto_arm_pro.pt"
learning_rate = 0.001
model = ANN()
model.load_state_dict(torch.load(PATH))

device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print(device)
optimizer = optim.SGD(model.parameters(), lr=learning_rate)
# Loss function.
# losses = nn.CrossEntropyLoss()
losses = nn.MSELoss()
loss_tr = []
loss_va = []

# evaluate a model using repeated k-fold cross-validation
for i in range(10000):
    train_loss = study(model, X_train,y_train, optimizer, losses, device)
    val_loss = val(model,X_val, y_val,losses, device)
    loss_tr.append(train_loss)
    loss_va.append(val_loss)

    if i%100==0:
        print('epoch: ',i, 'Train loss: ', train_loss)
        print('       ',i, 'Val loss: ', val_loss)

torch.save(model.state_dict(), PATH)

In [ ]:
import matplotlib.pyplot as plt

plt.title('Sai số góc khớp')
plt.xlabel('loss')
plt.ylabel('epoch')

plt.plot(loss_tr, label='train_loss')
plt.plot(loss_va, label='val_loss')
plt.legend()

plt.show()

# Visualize